In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from datetime import datetime, date

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/kpmg-data-set/KPMG_VI_New_raw_data_update_final (1).xlsx


In [2]:
trans_df = pd.read_excel('/kaggle/input/kpmg-data-set/KPMG_VI_New_raw_data_update_final (1).xlsx', sheet_name='Transactions')
Caddress_df = pd.read_excel('/kaggle/input/kpmg-data-set/KPMG_VI_New_raw_data_update_final (1).xlsx', sheet_name='CustomerAddress')
Cdemo_df = pd.read_excel('/kaggle/input/kpmg-data-set/KPMG_VI_New_raw_data_update_final (1).xlsx', sheet_name='CustomerDemographic')
NewC_df = pd.read_excel('/kaggle/input/kpmg-data-set/KPMG_VI_New_raw_data_update_final (1).xlsx', sheet_name='NewCustomerList')

In [3]:
def dataframe_set_header(df):
    df.columns = df.iloc[0]  # convert the first row to columns header
    df = df.drop(labels=[0], axis=0) # remove the first row
    return df

In [4]:
trans_df = dataframe_set_header(trans_df)
Caddress_df = dataframe_set_header(Caddress_df)
Cdemo_df = dataframe_set_header(Cdemo_df)
NewC_df = dataframe_set_header(NewC_df)

In [5]:
Cdemo_df.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


In [6]:
Caddress_df.columns

Index(['customer_id', 'address', 'postcode', 'state', 'country',
       'property_valuation'],
      dtype='object', name=0)

In [7]:
Caddress_df.head()

,customer_id,address,postcode,state,country,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,Australia,10
2,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
3,4,0 Holy Cross Court,4211,QLD,Australia,9
4,5,17979 Del Mar Point,2448,New South Wales,Australia,4
5,6,9 Oakridge Court,3216,VIC,Australia,9


In [8]:
Caddress_df[Caddress_df['country'] != 'Australia']

,customer_id,address,postcode,state,country,property_valuation


The only value in the 'country' column is 'Australia', so we can remove thie column.

In [9]:
Caddress_df = Caddress_df.drop(labels=['country'], axis=1)

In [10]:
Caddress_df

,customer_id,address,postcode,state,property_valuation
1,1,060 Morning Avenue,2016,New South Wales,10
2,2,6 Meadow Vale Court,2153,New South Wales,10
3,4,0 Holy Cross Court,4211,QLD,9
4,5,17979 Del Mar Point,2448,New South Wales,4
5,6,9 Oakridge Court,3216,VIC,9
...,...,...,...,...,...
3995,3999,1482 Hauk Trail,3064,VIC,3
3996,4000,57042 Village Green Point,4511,QLD,6
3997,4001,87 Crescent Oaks Alley,2756,NSW,10
3998,4002,8194 Lien Street,4032,QLD,7


In [11]:
Caddress_df['state'].value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

we have here some values written in full name in the 'state' column. So we will change them to the abbreviated form so as to have coherent names.

In [12]:
Caddress_df['state'] = Caddress_df['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC'})
Caddress_df.head()

,customer_id,address,postcode,state,property_valuation
1,1,060 Morning Avenue,2016,NSW,10
2,2,6 Meadow Vale Court,2153,NSW,10
3,4,0 Holy Cross Court,4211,QLD,9
4,5,17979 Del Mar Point,2448,NSW,4
5,6,9 Oakridge Court,3216,VIC,9


In [13]:
Caddress_df['state'].value_counts()

NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

we can also drop the 'address' and 'postcode' from the dataframe beacuse they are not expected to affect data analysis.

In [14]:
Caddress_df = Caddress_df.drop(labels=['address', 'postcode'], axis=1)

In [15]:
Caddress_df.head()

,customer_id,state,property_valuation
1,1,NSW,10
2,2,NSW,10
3,4,QLD,9
4,5,NSW,4
5,6,VIC,9


In [16]:
Caddress_df.describe()

,customer_id,state,property_valuation
count,3999,3999,3999
unique,3999,3,12
top,1,NSW,9
freq,1,2140,647


# CustomerDemographic dataframe:

In [17]:
Cdemo_df.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
1,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
2,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
3,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
4,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
5,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


In [18]:
Cdemo_df.columns

Index(['customer_id', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'DOB', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'default', 'owns_car', 'tenure'],
      dtype='object', name=0)

In [19]:
Cdemo_df['deceased_indicator'].value_counts()

N    3998
Y       2
Name: deceased_indicator, dtype: int64

In [20]:
Cdemo_df = Cdemo_df.loc[Cdemo_df['deceased_indicator'] == 'N']

now we will delete the 'deceased_indicator' column. We will remove the 'default' column too because it does not contain meaningfull data. Also the 'first_name' and 'last_name' columns wil be removed since they can't have any impact on the data analysis.

In [21]:
Cdemo_df = Cdemo_df.drop(labels=['deceased_indicator', 'default', 'first_name', 'last_name'], axis=1)

In [22]:
Cdemo_df.head()

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure
1,1,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,Yes,11
2,2,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,Yes,16
3,3,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,Yes,15
4,4,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,No,7
5,5,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,Yes,8


In [23]:
Cdemo_df['gender'].value_counts()

Female    2036
Male      1871
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [24]:
Cdemo_df['gender'] = Cdemo_df['gender'].replace({'M': 'Male', 'F': 'Female', 'Femal': 'Female', 'U': 'Unkown'})

In [25]:
Cdemo_df['gender'].value_counts()

Female    2038
Male      1872
Unkown      88
Name: gender, dtype: int64

Now we will extract age from DOB

In [26]:
Cdemo_df['DOB'].value_counts()

1978-01-30    7
1964-07-08    4
1962-12-17    4
1978-08-19    4
1977-05-13    4
             ..
1989-06-16    1
1998-09-30    1
1985-03-11    1
1989-10-23    1
1991-11-05    1
Name: DOB, Length: 3446, dtype: int64

In [27]:
Cdemo_df['DOB'] = pd.to_datetime(Cdemo_df['DOB']) # convert the type in this column to datetime

In [28]:
# source of this function: https://www.geeksforgeeks.org/convert-birth-date-to-age-in-pandas/
def age(born):
    #born = datetime.strptime(born, "%d/%m/%Y").date()
    today = date.today()
    return today.year - born.year - ((today.month,          # comparison using tuples
                                      today.day) < (born.month, 
                                                    born.day))

In [29]:
Cdemo_df['DOB'].dtype

dtype('<M8[ns]')

In [30]:
Cdemo_df.loc[Cdemo_df.isna().all(axis=1)]

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure


In [31]:
#Cdemo_df['age'].loc[Cdemo_df['age'].isna()]

In [32]:
Cdemo_df['age'] = Cdemo_df['DOB'].apply(age)

In [33]:
Cdemo_df.head()

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure,age
1,1,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11,69.0
2,2,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,Yes,16,42.0
3,3,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,Yes,15,69.0
4,4,Male,33,1961-10-03,NaN,IT,Mass Customer,No,7,61.0
5,5,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,Yes,8,46.0


In [34]:
Cdemo_df['age'].value_counts(ascending=True)

82.0       1
91.0       1
87.0       1
179.0      1
79.0       2
21.0      24
39.0      34
32.0      39
22.0      39
40.0      44
69.0      45
62.0      51
31.0      56
68.0      58
52.0      58
41.0      59
23.0      60
60.0      60
34.0      61
67.0      61
29.0      62
26.0      63
59.0      64
66.0      64
65.0      65
54.0      68
38.0      68
55.0      68
50.0      69
56.0      70
24.0      71
30.0      72
57.0      73
64.0      73
27.0      76
53.0      77
61.0      79
35.0      79
25.0      80
51.0      81
63.0      81
58.0      82
33.0      86
28.0      90
36.0      93
37.0     101
43.0     110
47.0     118
42.0     122
48.0     138
49.0     142
44.0     165
46.0     173
45.0     233
Name: age, dtype: int64

In [35]:
Cdemo_df

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure,age
1,1,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11,69.0
2,2,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,Yes,16,42.0
3,3,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,Yes,15,69.0
4,4,Male,33,1961-10-03,NaN,IT,Mass Customer,No,7,61.0
5,5,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,Yes,8,46.0
...,...,...,...,...,...,...,...,...,...,...
3996,3996,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,No,19,47.0
3997,3997,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,Yes,1,21.0
3998,3998,Unkown,60,NaT,Assistant Manager,IT,High Net Worth,No,NaN,NaN
3999,3999,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,Yes,10,49.0


In [36]:
Cdemo_df.dtypes

0
customer_id                                    object
gender                                         object
past_3_years_bike_related_purchases            object
DOB                                    datetime64[ns]
job_title                                      object
job_industry_category                          object
wealth_segment                                 object
owns_car                                       object
tenure                                         object
age                                           float64
dtype: object

In [37]:
# we have a wrong value age = 179: we will remove this column
Cdemo_df = Cdemo_df.loc[Cdemo_df['age'] != 179]

In [38]:
max(Cdemo_df['age'])

91.0

In [39]:
Cdemo_df['age'].value_counts()

45.0    233
46.0    173
44.0    165
49.0    142
48.0    138
42.0    122
47.0    118
43.0    110
37.0    101
36.0     93
28.0     90
33.0     86
58.0     82
51.0     81
63.0     81
25.0     80
61.0     79
35.0     79
53.0     77
27.0     76
57.0     73
64.0     73
30.0     72
24.0     71
56.0     70
50.0     69
55.0     68
54.0     68
38.0     68
65.0     65
59.0     64
66.0     64
26.0     63
29.0     62
34.0     61
67.0     61
60.0     60
23.0     60
41.0     59
68.0     58
52.0     58
31.0     56
62.0     51
69.0     45
40.0     44
32.0     39
22.0     39
39.0     34
21.0     24
79.0      2
91.0      1
87.0      1
82.0      1
Name: age, dtype: int64

In [40]:
#from pandasql import sqldf
#sql_query = "SELECT customer_id, age FROM Cdemo_df WHERE age < 40;"
#print(sqldf(sql_query, locals()))

In [41]:
# create age categories: AgeGroup

Cdemo_df['AgeGroup'] = "" # add an empty column to the dataframe
Cdemo_df['AgeGroup'][ Cdemo_df.age < 100 ] = 'older' # 91 is the max age in this dataset
Cdemo_df['AgeGroup'][ Cdemo_df.age < 50 ] = 'fifties'
Cdemo_df['AgeGroup'][ Cdemo_df.age < 50 ] = 'forties'
Cdemo_df['AgeGroup'][ Cdemo_df.age < 40 ] = 'thirties'
Cdemo_df['AgeGroup'][ Cdemo_df.age < 30 ] = 'twenties'

/tmp/ipykernel_21/80504482.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cdemo_df['AgeGroup'] = "" # add an empty column to the dataframe
/tmp/ipykernel_21/80504482.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Cdemo_df['AgeGroup'][ Cdemo_df.age < 100 ] = 'older' # 91 is the max age in this dataset
/tmp/ipykernel_21/80504482.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [42]:
# create age categories: AgeGroup

#Cdemo_df['age'][ Cdemo_df.age < 40 ] = 'older'
#Cdemo_df['age'][ Cdemo_df.age < 50 ] = 'fifties'
#Cdemo_df['age'][ Cdemo_df.age < 50 ] = 'forties'
#Cdemo_df['age'][ Cdemo_df.age < 40 ] = 'thirties'
#Cdemo_df['age'][ Cdemo_df.age < 30 ] = 'twenties'

# rename the column

#Cdemo_df.rename(columns={'age': 'AgeGroup'}, inplace= True)

In [43]:
Cdemo_df

,customer_id,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,owns_car,tenure,age,AgeGroup
1,1,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,Yes,11,69.0,older
2,2,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,Yes,16,42.0,forties
3,3,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,Yes,15,69.0,older
4,4,Male,33,1961-10-03,NaN,IT,Mass Customer,No,7,61.0,older
5,5,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,Yes,8,46.0,forties
...,...,...,...,...,...,...,...,...,...,...,...
3996,3996,Female,8,1975-08-09,VP Product Management,Health,Mass Customer,No,19,47.0,forties
3997,3997,Female,87,2001-07-13,Statistician II,Manufacturing,High Net Worth,Yes,1,21.0,twenties
3998,3998,Unkown,60,NaT,Assistant Manager,IT,High Net Worth,No,NaN,NaN,
3999,3999,Male,11,1973-10-24,NaN,Manufacturing,Affluent Customer,Yes,10,49.0,forties


In [44]:
Cdemo_df['AgeGroup'].value_counts()

older       1352
forties     1304
thirties     689
twenties     565
              87
Name: AgeGroup, dtype: int64

I will keep the 'age' column for now in case I need to change the age categories, but I will delete the 'DOB' column

In [45]:
Cdemo_df = Cdemo_df.drop(labels=['DOB'], axis=1)
Cdemo_df

,customer_id,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,owns_car,tenure,age,AgeGroup
1,1,Female,93,Executive Secretary,Health,Mass Customer,Yes,11,69.0,older
2,2,Male,81,Administrative Officer,Financial Services,Mass Customer,Yes,16,42.0,forties
3,3,Male,61,Recruiting Manager,Property,Mass Customer,Yes,15,69.0,older
4,4,Male,33,NaN,IT,Mass Customer,No,7,61.0,older
5,5,Female,56,Senior Editor,NaN,Affluent Customer,Yes,8,46.0,forties
...,...,...,...,...,...,...,...,...,...,...
3996,3996,Female,8,VP Product Management,Health,Mass Customer,No,19,47.0,forties
3997,3997,Female,87,Statistician II,Manufacturing,High Net Worth,Yes,1,21.0,twenties
3998,3998,Unkown,60,Assistant Manager,IT,High Net Worth,No,NaN,NaN,
3999,3999,Male,11,NaN,Manufacturing,Affluent Customer,Yes,10,49.0,forties


# Transactions table:

In [46]:
trans_df.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
5,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [47]:
#trans_df['product_first_sold_date'] = pd.to_datetime(trans_df['product_first_sold_date']).dt.date

In [48]:
trans_df['transaction_date'] = pd.to_datetime(trans_df['transaction_date']).dt.date

In [49]:
trans_df

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
5,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.4,37823
19997,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74,35560
19998,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.9,44.71,40410
19999,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216


In [50]:
trans_df['product_first_sold_date'].value_counts()# I should handle the problem with these dates.....

33879    234
41064    229
37823    227
39880    222
38216    220
        ... 
41848    169
42404    168
41922    166
37659    163
34586    162
Name: product_first_sold_date, Length: 100, dtype: int64

In [51]:
trans_df

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
2,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
3,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
4,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
5,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.4,37823
19997,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74,35560
19998,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.9,44.71,40410
19999,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,38216


In [52]:
import datetime
def xldate_to_datetime(xldate):
    temp = datetime.datetime(1900, 1, 1)
    delta = datetime.timedelta(days=xldate)
    return temp+delta

In [53]:
xldate_to_datetime(35560)

datetime.datetime(1997, 5, 12, 0, 0)

In [54]:
trans_df = trans_df.loc[trans_df['product_first_sold_date'].notna()]

In [55]:
trans_df['product_first_sold_date'].value_counts()

33879    234
41064    229
37823    227
39880    222
38216    220
        ... 
41848    169
42404    168
41922    166
37659    163
34586    162
Name: product_first_sold_date, Length: 100, dtype: int64

In [56]:
trans_df['product_first_sold_date'] = trans_df['product_first_sold_date'].apply(lambda x: xldate_to_datetime(x))

/tmp/ipykernel_21/741511739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trans_df['product_first_sold_date'] = trans_df['product_first_sold_date'].apply(lambda x: xldate_to_datetime(x))


In [57]:
trans_df

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
1,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,2012-12-04
2,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,2014-03-05
3,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,1999-07-22
4,4,88,3135,2017-08-31,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,1998-12-18
5,5,78,787,2017-10-01,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,2015-08-12
...,...,...,...,...,...,...,...,...,...,...,...,...,...
19996,19996,51,1018,2017-06-24,True,Approved,OHM Cycles,Standard,high,medium,2005.66,1203.4,2003-07-23
19997,19997,41,127,2017-11-09,True,Approved,Solex,Road,medium,medium,416.98,312.74,1997-05-12
19998,19998,87,2284,2017-04-14,True,Approved,OHM Cycles,Standard,medium,medium,1636.9,44.71,2010-08-22
19999,19999,6,2764,2017-07-03,False,Approved,OHM Cycles,Standard,high,medium,227.88,136.73,2004-08-19


the column 'product_first_sold_date' contains NaN values that is why the conversion is not working.
So I removed them and it worked.

Here I save the cleaned dataframes as excel files.

In [58]:
Cdemo_df.to_excel('Customer_demographics_cleaned1.xlsx', index=False)
trans_df.to_excel('Transactions_cleaned1.xlsx', index=False)

In [59]:
Caddress_df.to_excel('Customer_address_cleaned1.xlsx', index=False)

# NewCostumerList dataset

In [60]:
NewC_df

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,NaN,NaN,NaN,NaN,NaN,Rank,Value
1,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.875000,0.743750,1.0,1,1.71875
2,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.89,0.8900,1.112500,0.945625,1.0,1,1.71875
3,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,1.01,1.0100,1.010000,1.010000,1.0,1,1.71875
4,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.87,1.0875,1.087500,1.087500,4.0,4,1.703125
5,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.52,0.5200,0.650000,0.650000,4.0,4,1.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,...,NSW,Australia,7,0.79,0.7900,0.790000,0.790000,996.0,996,0.374
997,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,...,NSW,Australia,10,0.76,0.7600,0.950000,0.807500,997.0,997,0.357
998,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,...,QLD,Australia,2,0.85,1.0625,1.062500,1.062500,997.0,997,0.357
999,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,...,QLD,Australia,2,1.09,1.3625,1.362500,1.158125,997.0,997,0.357


In [61]:
NewC_df['country'].value_counts()

Australia    1000
Name: country, dtype: int64

In [62]:
NewC_df['state'].value_counts()

NSW    506
VIC    266
QLD    228
Name: state, dtype: int64

In [63]:
NewC_df = NewC_df[NewC_df.columns.dropna()]

In [64]:
NewC_df

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
1,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.71875
2,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.71875
3,Ardelis,Forrester,Female,10,1974-08-28 00:00:00,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.71875
4,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
5,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996,Ferdinand,Romanetti,Male,60,1959-10-07,Paralegal,Financial Services,Affluent Customer,N,No,9,2 Sloan Way,2200,NSW,Australia,7,996,0.374
997,Burk,Wortley,Male,22,2001-10-17,Senior Sales Associate,Health,Mass Customer,N,No,6,04 Union Crossing,2196,NSW,Australia,10,997,0.357
998,Melloney,Temby,Female,17,1954-10-05,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,N,Yes,15,33475 Fair Oaks Junction,4702,QLD,Australia,2,997,0.357
999,Dickie,Cubbini,Male,30,1952-12-17,Financial Advisor,Financial Services,Mass Customer,N,Yes,19,57666 Victoria Way,4215,QLD,Australia,2,997,0.357


In [65]:
#
NewC_df = NewC_df.drop(labels=['country'], axis=1)
#Caddress_df['state'] = Caddress_df['state'].replace({'New South Wales': 'NSW', 'Victoria': 'VIC'})
#Caddress_df = Caddress_df.drop(labels=['address', 'postcode'], axis=1)

NewC_df = NewC_df.loc[NewC_df['deceased_indicator'] == 'N']
NewC_df = NewC_df.drop(labels=['deceased_indicator', 'Value', 'first_name', 'last_name'], axis=1)
NewC_df['gender'] = NewC_df['gender'].replace({'M': 'Male', 'F': 'Female', 'Femal': 'Female', 'U': 'Unkown'})
NewC_df['DOB'] = pd.to_datetime(NewC_df['DOB']) # convert the type in this column to datetime
NewC_df['age'] = NewC_df['DOB'].apply(age)
# create age categories: AgeGroup

NewC_df['AgeGroup'] = "" # add an empty column to the dataframe
NewC_df['AgeGroup'][ NewC_df.age < 100 ] = 'older' # 91 is the max age in this dataset
NewC_df['AgeGroup'][ NewC_df.age < 50 ] = 'fifties'
NewC_df['AgeGroup'][ NewC_df.age < 50 ] = 'forties'
NewC_df['AgeGroup'][ NewC_df.age < 40 ] = 'thirties'
NewC_df['AgeGroup'][ NewC_df.age < 30 ] = 'twenties'
NewC_df = NewC_df.drop(labels=['DOB'], axis=1)

/tmp/ipykernel_21/85387625.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewC_df['AgeGroup'][ NewC_df.age < 100 ] = 'older' # 91 is the max age in this dataset
/tmp/ipykernel_21/85387625.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewC_df['AgeGroup'][ NewC_df.age < 50 ] = 'fifties'
/tmp/ipykernel_21/85387625.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  NewC_df['AgeGroup'][ NewC_df.age < 50 ] = 'forties'
/tmp/ipy

In [66]:
NewC_df = NewC_df.drop(labels=['address', 'postcode'], axis=1)

In [67]:
NewC_df

,gender,past_3_years_bike_related_purchases,job_title,job_industry_category,wealth_segment,owns_car,tenure,state,property_valuation,Rank,age,AgeGroup
1,Male,86,General Manager,Manufacturing,Mass Customer,Yes,14,QLD,6,1,65.0,older
2,Male,69,Structural Engineer,Property,Mass Customer,No,16,NSW,11,1,53.0,older
3,Female,10,Senior Cost Accountant,Financial Services,Affluent Customer,No,10,VIC,5,1,48.0,forties
4,Female,64,Account Representative III,Manufacturing,Affluent Customer,Yes,5,QLD,1,4,44.0,forties
5,Female,34,Financial Analyst,Financial Services,Affluent Customer,No,19,NSW,9,4,57.0,older
...,...,...,...,...,...,...,...,...,...,...,...,...
996,Male,60,Paralegal,Financial Services,Affluent Customer,No,9,NSW,7,996,63.0,older
997,Male,22,Senior Sales Associate,Health,Mass Customer,No,6,NSW,10,997,21.0,twenties
998,Female,17,Budget/Accounting Analyst IV,Financial Services,Affluent Customer,Yes,15,QLD,2,997,68.0,older
999,Male,30,Financial Advisor,Financial Services,Mass Customer,Yes,19,QLD,2,997,70.0,older


In [68]:
NewC_df.to_excel('New_Customer_cleaned1.xlsx', index=False)